Prediction Candidate Identification
===



In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
import os
import json
import sys
import pickle
from tqdm import tqdm

import sklearn
import sklearn.linear_model
import sklearn.preprocessing
from sklearn.pipeline import Pipeline

import dateutil.parser
from dateutil.relativedelta import relativedelta
from datetime import datetime, timedelta
import pytz

In [ ]:
# HuggingFace packages
import transformers
import tokenizers
import torch

# more torch imports
import torchvision
import torchvision.transforms
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
from pathlib import Path
git_root_dir = !git rev-parse --show-toplevel
git_root_dir = Path(git_root_dir[0].strip())
git_root_dir

In [ ]:
import sys
sys.path.append(os.path.join(git_root_dir, 'src'))
import cbrec.genconfig

In [ ]:
config = cbrec.genconfig.Config()
#config.metadata_filepath += "_old"
#config.feature_db_filepath += "_old"

In [ ]:
import cbrec.featuredb
import cbrec.utils
import cbrec.data
import cbrec.reccontext
import cbrec.evaluation
import cbrec.torchmodel
import cbrec.text.embeddingdb
import cbrec.text.journalid

In [ ]:
import cbrec.logutils
cbrec.logutils.set_up_logging()

### Extracting from the RecDataGenerator instance

In the interactive mode...

```
# get the time from which to retrieve journals
next(iter(rdg.journal_dict.od)) - 1
```

Could use the checkpoint at the end of the list to generate this. However, this file is now generated as a standard part of the predict.py script.

```
In [2]: active_user_ids = rdg.activity_manager.get_active_user_ids()                                                                     
                                                                                                                                         
In [3]: len(active_user_ids)                                                                                                             
Out[3]: 14169                                                                                                                            
                                                                                                                                         
In [4]: eligible_user_ids = rdg.eligibility_manager.get_eligible_user_ids().copy()                                                       
                                                                                                                                         
In [5]: len(eligible_user_ids)                                                                                                           
Out[5]: 473365                                                                                                                           
                                                                                                                                         
In [6]: eligible_active_user_ids = active_user_ids & eligible_user_ids                                                                   
                                                                                                                                         
In [7]: len(eligible_active_user_ids)                                                                                                    
Out[7]: 12713

...

In [18]: candidate_usps = []                                                                                                             
                                                                                                                                         
In [19]: for candidate_user_id in eligible_active_user_ids:                                                                              
    ...:     for site_id in rdg.eligibility_manager.get_eligible_sites_from_user(candidate_user_id):                                     
    ...:         candidate_usps.append((candidate_user_id, site_id))                                                                     
    ...:                                                                                                                                 
                                                                                                                                         
In [20]: len(candidate_usps)
Out[20]: 13703

In [21]: with open("prediction_usps.tsv", 'w') as outfile:
    ...:     for candidate_usp in candidate_usps:
    ...:         outfile.write(f"{candidate_usp[0]}\t{candidate_usp[1]}\n")
    ...:

```

In [ ]:
dm = cbrec.data.DataManager(config, load_ints=False, load_journals=True)

In [ ]:
journal_df = dm.get_filtered_journals()
len(journal_df)

In [ ]:
jil = cbrec.text.journalid.JournalIdLookup(config, dm)

In [ ]:
# this filepath essentially comes straight from the predict.py output
# e.g. 2021-09-15 23:40:03,033 - cbrec.prediction.main - INFO - Wrote 5965 candidate USPs to '/home/lana/shared/caringbridge/data/projects/recsys-peer-match/feature_data/checkpoints/rdg_36028292_734622_prediction_usps.tsv'.                                                          
candidate_usps = []
candidate_usp_filepath = os.path.join(config.checkpoint_dir, f"rdg_36261144_851080_prediction_usps.tsv")
with open(candidate_usp_filepath, 'r') as infile:
    for line in infile:
        line = line.strip()
        if line != '':
            user_id, site_id = line.split("\t")
            user_id, site_id = int(user_id), int(site_id)
            candidate_usps.append((user_id, site_id))
len(candidate_usps)

In [ ]:
# this should be whatever timestamp is used with predict.py (inferred or not)
timestamp = 1633029219763

In [ ]:
journal_oids = []
n_not_enough = 0
for candidate_usp in candidate_usps:
    journal_updates_before = jil.get_journal_updates_before(candidate_usp, timestamp + 1)
    if len(journal_updates_before) < 3:
        n_not_enough += 1
    else:
        journal_oids.extend(journal_updates_before)
        
len(journal_oids), n_not_enough

In [ ]:
journal_oids = set(journal_oids)
len(journal_oids)

In [ ]:
with open(os.path.join(config.model_data_dir, 'predict_candidate_journal_oids.txt'), 'w') as outfile:
    for journal_oid in journal_oids:
        outfile.write(journal_oid + "\n")